In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
df=pd.read_csv('spam.csv')

In [3]:
df

,Category,Message
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."
...,...,...
5567,spam,This is the 2nd time we have tried 2 contact u...
5568,ham,Will ü b going to esplanade fr home?
5569,ham,"Pity, * was in mood for that. So...any other s..."
5570,ham,The guy did some bitching but I acted like i'd...


In [4]:
df.isnull().sum()

Category    0
Message     0
dtype: int64

In [5]:
df.duplicated().sum()

415

In [6]:
df.drop_duplicates(inplace=True,ignore_index=True)

In [7]:
df

,Category,Message
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."
...,...,...
5152,spam,This is the 2nd time we have tried 2 contact u...
5153,ham,Will ü b going to esplanade fr home?
5154,ham,"Pity, * was in mood for that. So...any other s..."
5155,ham,The guy did some bitching but I acted like i'd...


In [8]:
X=df.drop('Category',axis=1)
y=df['Category']

In [9]:
import nltk
import re
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords

In [10]:
lem=WordNetLemmatizer()

In [11]:
corpus=[]
for i in range(len(df)):
    sentence=re.sub('[^a-zA-Z]',' ',df['Message'][i])
    sentence=sentence.lower()
    words=sentence.split()  #Tokenize
    words=[lem.lemmatize(word)for word in words if word not in set(stopwords.words('english'))]    #Stemming
    sentence=' '.join(words)
    corpus.append(sentence)

In [12]:
corpus

['go jurong point crazy available bugis n great world la e buffet cine got amore wat',
 'ok lar joking wif u oni',
 'free entry wkly comp win fa cup final tkts st may text fa receive entry question std txt rate c apply',
 'u dun say early hor u c already say',
 'nah think go usf life around though',
 'freemsg hey darling week word back like fun still tb ok xxx std chgs send rcv',
 'even brother like speak treat like aid patent',
 'per request melle melle oru minnaminunginte nurungu vettam set callertune caller press copy friend callertune',
 'winner valued network customer selected receivea prize reward claim call claim code kl valid hour',
 'mobile month u r entitled update latest colour mobile camera free call mobile update co free',
 'gonna home soon want talk stuff anymore tonight k cried enough today',
 'six chance win cash pound txt csh send cost p day day tsandcs apply reply hl info',
 'urgent week free membership prize jackpot txt word claim c www dbuk net lccltd pobox ldnw rw'

# Onehot Representation

In [13]:
from tensorflow.keras.layers import Embedding
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Bidirectional
from tensorflow.keras.layers import Dropout

In [14]:
### Vocabulary size
voc_size=5000

In [15]:
onehot_repr=[one_hot(words,voc_size)for words in corpus] 
onehot_repr

[[1421,
  1293,
  1536,
  3034,
  2836,
  553,
  4846,
  327,
  2891,
  4224,
  1745,
  2360,
  729,
  4961,
  2236,
  3221],
 [2859, 3573, 614, 1618, 175, 2449],
 [3157,
  860,
  3712,
  2636,
  4147,
  53,
  1774,
  2220,
  2978,
  2377,
  3766,
  822,
  53,
  3434,
  860,
  4228,
  4301,
  402,
  1193,
  1419,
  2203],
 [175, 2528, 3269, 4067, 1318, 175, 1419, 2000, 3269],
 [1220, 532, 1421, 1164, 1975, 3834, 61],
 [405,
  3040,
  2973,
  3306,
  2689,
  41,
  3316,
  4190,
  2287,
  742,
  2859,
  886,
  4301,
  4427,
  3181,
  3343],
 [2612, 3319, 3316, 1244, 4233, 3316, 4072, 4453],
 [943,
  4825,
  1708,
  1708,
  2426,
  872,
  4225,
  1856,
  554,
  4983,
  4444,
  4076,
  2946,
  1639,
  4983],
 [4677,
  4002,
  945,
  3027,
  3486,
  277,
  604,
  445,
  2098,
  621,
  2098,
  1290,
  3642,
  4885,
  3602],
 [1943,
  2714,
  175,
  4441,
  2022,
  775,
  4807,
  2824,
  1943,
  3956,
  3157,
  621,
  1943,
  775,
  49,
  3157],
 [940, 4238, 1909, 4878, 4924, 3116, 3330, 1108

# Embedding Representation

In [16]:
sent_length=20
embedded_docs=pad_sequences(onehot_repr,padding='pre',maxlen=sent_length)
print(embedded_docs)
print(embedded_docs.shape)

[[   0    0    0 ... 4961 2236 3221]
 [   0    0    0 ... 1618  175 2449]
 [ 860 3712 2636 ... 1193 1419 2203]
 ...
 [   0    0    0 ...  656 2576 4825]
 [   0    0    0 ... 4881  175 3157]
 [   0    0    0 ... 2615 1695 4539]]
(5157, 20)


In [17]:
from sklearn.preprocessing import LabelEncoder
le=LabelEncoder()

In [18]:
X_final=np.array(embedded_docs)
y_final=le.fit_transform(y)

In [19]:
print(X_final.shape)
print(y_final.shape)

(5157, 20)
(5157,)


In [20]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_final, y_final, test_size=0.20, random_state=11)

In [21]:
## Creating model
embedding_vector_features=40
model=Sequential()
model.add(Embedding(voc_size,embedding_vector_features,input_length=sent_length))
model.add(LSTM(100))
model.add(Dense(1,activation='sigmoid'))
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
print(model.summary())

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 20, 40)            200000    
                                                                 
 lstm (LSTM)                 (None, 100)               56400     
                                                                 
 dense (Dense)               (None, 1)                 101       
                                                                 
Total params: 256,501
Trainable params: 256,501
Non-trainable params: 0
_________________________________________________________________
None


# Model Training

In [22]:
model.fit(X_train,y_train,validation_data=(X_test,y_test),epochs=10,batch_size=64)

Epoch 1/10
65/65 [==============================] - 6s 45ms/step - loss: 0.3055 - accuracy: 0.8950 - val_loss: 0.2019 - val_accuracy: 0.9399
Epoch 2/10
65/65 [==============================] - 2s 35ms/step - loss: 0.0814 - accuracy: 0.9811 - val_loss: 0.0784 - val_accuracy: 0.9767
Epoch 3/10
65/65 [==============================] - 2s 34ms/step - loss: 0.0343 - accuracy: 0.9905 - val_loss: 0.0720 - val_accuracy: 0.9787
Epoch 4/10
65/65 [==============================] - 2s 28ms/step - loss: 0.0194 - accuracy: 0.9964 - val_loss: 0.0677 - val_accuracy: 0.9767
Epoch 5/10
65/65 [==============================] - 2s 30ms/step - loss: 0.0130 - accuracy: 0.9973 - val_loss: 0.0769 - val_accuracy: 0.9767
Epoch 6/10
65/65 [==============================] - 2s 33ms/step - loss: 0.0121 - accuracy: 0.9971 - val_loss: 0.0683 - val_accuracy: 0.9826
Epoch 7/10
65/65 [==============================] - 2s 34ms/step - loss: 0.0112 - accuracy: 0.9971 - val_loss: 0.0846 - val_accuracy: 0.9767
Epoch 8/10
65

# Performance Metrics And Accuracy

In [23]:
y_pred=model.predict(X_test)

33/33 [==============================] - 1s 8ms/step


In [24]:
y_pred=(y_pred>0.5)

In [25]:
from sklearn.metrics import confusion_matrix,accuracy_score,classification_report

In [26]:
confusion_matrix(y_pred,y_test)

array([[894,  16],
       [  7, 115]], dtype=int64)

In [27]:
print(classification_report(y_pred,y_test))

              precision    recall  f1-score   support

       False       0.99      0.98      0.99       910
        True       0.88      0.94      0.91       122

    accuracy                           0.98      1032
   macro avg       0.94      0.96      0.95      1032
weighted avg       0.98      0.98      0.98      1032



In [28]:
accuracy_score(y_test,y_pred)

0.9777131782945736